In [31]:
from pprint import pprint 
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'E_commarce.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

# Django Filters module 

Documentation
https://django-filter.readthedocs.io/en/stable/

In [32]:
from blogging_app.models import * 

In [33]:
# yeah tho request create karne k liye h json formate mein

import json 

from rest_framework.request import Request
from rest_framework.test import APIRequestFactory
from rest_framework.parsers import JSONParser


factory = APIRequestFactory()

In [43]:
from django_filters.rest_framework import FilterSet, CharFilter


class PostFilter(FilterSet):
    """
    A filterset class for filtering and ordering Task records
    """

    user = CharFilter(method='filter_with_user')

    def filter_with_user(self, queryset, name, value):
        print("filter_with_user method is called with ( name:", name,"value:", value, ")")
        return queryset.filter(
            user__id=value
        )

    class Meta:
        model = Post
        fields = [
            'user',
        ]

In [44]:
request_instance = Request(
    request=factory.get(
        path='/posts/?user=3',
        data={},
        format='json'
    ),
    parsers=[JSONParser()],
)

print("-"*100)
print("JSON data sent in the request:\n", request_instance.data, "\n")
print("Query Params sent in the request:\n",request_instance.query_params, "\n")
print("-"*100)

----------------------------------------------------------------------------------------------------
JSON data sent in the request:
 {} 

Query Params sent in the request:
 <QueryDict: {'user': ['3']}> 

----------------------------------------------------------------------------------------------------


In [36]:
filterset = PostFilter(
    queryset=Post.objects.all(),
    data=request_instance.query_params,
)

filtered_qs = filterset.qs

print(filtered_qs)

filter_with_user method is called with ( name: user value: 3 )
<QuerySet [<Post: Cycling Across Morocco, From the High Atlas to the>]>


In [37]:
class PostFilter(FilterSet):
    """
    A filterset class for filtering and ordering Task records
    """

    created_by = CharFilter(method='filter_with_user')

    def filter_with_user(self, queryset, name, value):
        print("filter_with_user method is called with ( name:", name,"value:", value, ")")
        user_ids = value.split(',')
        return queryset.filter(
            user__id__in=user_ids
        )

    class Meta:
        model = Post
        fields = [
            'created_by'
        ]


In [38]:
request_instance = Request(
        request=factory.get(
            path='/posts/?created_by=1,2',  # The created_by query param can have multiple values
            format='json'
        ),
    parsers=[JSONParser()],
)

print("-"*100)
print("JSON data sent in the request:\n", request_instance.data, "\n")
print("Query Params sent in the request:\n",request_instance.query_params, "\n")
print("-"*100)

filterset = PostFilter(
    queryset=Post.objects.all(),
    data=request_instance.query_params,
)

filtered_qs = filterset.qs

print(filtered_qs)

----------------------------------------------------------------------------------------------------
JSON data sent in the request:
 {} 

Query Params sent in the request:
 <QueryDict: {'created_by': ['1,2']}> 

----------------------------------------------------------------------------------------------------
filter_with_user method is called with ( name: created_by value: 1,2 )
<QuerySet [<Post: vibes of nature>, <Post: Amaravati will be Andhra capital, no ‘3-capitals’>, <Post: Atlas to the>, <Post: White Outfit>, <Post: rrg rggetlas to the>]>


In [45]:
class PostFilter(FilterSet):
    """
    A filterset class for filtering and ordering Task records
    """

    created_by = CharFilter(method='filter_with_user')
    categories = CharFilter(method='filter_with_categories')

    def filter_with_user(self, queryset, name, value):
        print("filter_with_user method is called with ( name:", name,"value:", value, ")")
        user_ids = value.split(',')
        return queryset.filter(
            user__id__in=user_ids
        )
    
    def filter_with_categories(self, queryset, name, value):
        print("filter_with_categories method is called with ( name:", name,"value:", value, ")")
        category_ids = value.split(',')
        return queryset.filter(
            category__id__in=category_ids
        )

    class Meta:
        model = Post
        fields = [
            'created_by',
            'categories',
        ]

In [49]:
request_instance = Request(
        request=factory.get(
            path='/posts/?categories=1&created_by=2', 
            format='json'
        ),
    parsers=[JSONParser()],
)

print("-"*100)
print("JSON data sent in the request:\n", request_instance.data, "\n")
print("Query Params sent in the request:\n",request_instance.query_params, "\n")
print("-"*100)


filterset = PostFilter(
    queryset=Post.objects.all(),
    data=request_instance.query_params,
)

filtered_qs = filterset.qs

print(filtered_qs)

----------------------------------------------------------------------------------------------------
JSON data sent in the request:
 {} 

Query Params sent in the request:
 <QueryDict: {'categories': ['1'], 'created_by': ['2']}> 

----------------------------------------------------------------------------------------------------
filter_with_user method is called with ( name: created_by value: 2 )
filter_with_categories method is called with ( name: categories value: 1 )
<QuerySet [<Post: rrg rggetlas to the>]>


In [41]:
class PostFilter(FilterSet):
    """
    A filterset class for filtering and ordering Task records
    """

    created_by = CharFilter(method='filter_with_user')
    categories = CharFilter(method='filter_with_categories')
    ordering = CharFilter(method='ordering_by_params')

    def filter_with_user(self, queryset, name, value):
        print("filter_with_user method is called with ( name:", name,"value:", value, ")")
        user_ids = value.split(',')
        return queryset.filter(
            user__id__in=user_ids
        )
    
    def filter_with_categories(self, queryset, name, value):
        print("filter_with_categories method is called with ( name:", name,"value:", value, ")")
        category_ids = value.split(',')
        return queryset.filter(
            category__id__in=category_ids
        )

    def ordering_by_params(self, queryset, name, value):
        print("ordering_by_params method is called with ( name:", name,"value:", value, ")")
        if value:
            ordering_list = value.split(',')
        else:
            ordering_list = ['id']
        return queryset.order_by(*ordering_list)

    class Meta:
        model = Post
        fields = [
            'created_by',
            'categories',
            'ordering'
        ]

In [42]:
request_instance = Request(
        request=factory.get(
            path='/posts/?ordering=-title', 
            format='json'
        ),
    parsers=[JSONParser()],
)

print("-"*100)
print("JSON data sent in the request:\n", request_instance.data, "\n")
print("Query Params sent in the request:\n",request_instance.query_params, "\n")
print("-"*100)

filterset = PostFilter(
    queryset=Post.objects.all(),
    data=request_instance.query_params,
)

filtered_qs = filterset.qs

print(filtered_qs)

----------------------------------------------------------------------------------------------------
JSON data sent in the request:
 {} 

Query Params sent in the request:
 <QueryDict: {'ordering': ['-title']}> 

----------------------------------------------------------------------------------------------------
ordering_by_params method is called with ( name: ordering value: -title )
<QuerySet [<Post: White Outfit>, <Post: vibes of nature>, <Post: rrg rggetlas to the>, <Post: Cycling Across Morocco, From the High Atlas to the>, <Post: capital, no ‘3-capitals’>, <Post: Avocado Caesar Green Beans>, <Post: Avocado Caesar>, <Post: Atlas to the>, <Post: A Simple Black & White Outfit>, <Post: Amaravati will be Andhra capital, no ‘3-capitals’>]>
